In [1]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

import numpy as np
from datetime import datetime
from packaging import version
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import TensorBoard

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.3.0 or above."

TensorFlow version:  2.3.0


In [2]:
img_input_data = keras.layers.Input(shape=(12, 128, 256), name="input_imgs")
permute1 = keras.layers.Permute((2,3,1))(img_input_data)
permute1 = tf.keras.layers.Conv2D(3, 3, padding='same')(permute1)
efficientnet = keras.applications.EfficientNetB2(include_top=False, weights = "imagenet", input_shape=(128,256,3))(permute1)
output = efficientnet
model = keras.models.Model(inputs=[img_input_data], outputs=[output])

# Renaming all layers which deals with image
"""
for layer in model.layers:
    layer._name = "vision_" + layer.name
"""
    
#concat = keras.layers.concatenate([input_data, hidden2])


print(model.summary())

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_imgs (InputLayer)      [(None, 12, 128, 256)]    0         
_________________________________________________________________
permute (Permute)            (None, 128, 256, 12)      0         
_________________________________________________________________
conv2d (Conv2D)              (None, 128, 256, 3)       327       
_________________________________________________________________
efficientnetb2 (Functional)  (None, 4, 8, 1408)        7768569   
Total params: 7,768,896
Trainable params: 7,701,321
Non-trainable params: 67,575
_________________________________________________________________
None
